In [1]:
import requests
import os
import pickle
from dotenv import load_dotenv

In [2]:
load_dotenv()
api_key = os.getenv("ANTHROPIC_API_KEY")

In [3]:
client = anthropic.Anthropic(
    api_key=api_key,
)

In [4]:
import base64
import httpx

In [5]:
with open("responses_openai.pkl", "rb") as file:
    responses = pickle.load(file)

In [6]:
urls = list(responses.keys())

In [7]:
image_media_type = "image/jpeg"

In [8]:
def encode_image(url):
    return base64.b64encode(httpx.get(url).content).decode("utf-8")

In [9]:
with open("/Users/damoncrockett/samplebooks/src/assets/txt/modelrank_prompt.txt", 'r') as f:
    prompt = f.read()

In [10]:
def get_response(url):
    message = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1024,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": image_media_type,
                            "data": encode_image(url),
                        },
                    },
                    {
                        "type": "text",
                        "text": prompt
                    }
                ],
            }
        ],
    )

    return message.content[0].text

In [11]:
responses_anthropic = {}

for url in urls:
    response = get_response(url)
    responses_anthropic[url] = {'claude-3-5-sonnet-20241022':response}

In [13]:
with open("responses_anthropic.pkl", "wb") as f:
    pickle.dump(responses_anthropic, f)